# Uzupełnianie przy użyciu sieci INR (ang. *Implicit Neural Representations*)

## Wstęp

W ostatnich latach sztuczna inteligencja coraz skuteczniej radzi sobie z przetwarzaniem obrazów i filmów – nie tylko rozpoznaje obiekty, ale także potrafi **uzupełniać brakujące fragmenty obrazu**, **kompresować dane** czy nawet **generować nowe klatki filmu**.

Wyobraź sobie, że masz do dyspozycji film, w którym część obrazu – np. prostokątny fragment w każdej klatce – została zasłonięta, tak jakby ktoś wyciął tę część nożyczkami lub zakrył ją taśmą. Twoim zadaniem jest **odtworzenie brakujących fragmentów tylko na podstawie współrzędnych piksela i czasu**.

W tym zadaniu skupimy się na bardziej zaawansowanym problemie: stworzeniu modelu całego filmu jako funkcji, która dla dowolnych współrzędnych piksela i numeru klatki zwraca jego kolor i przynależność do obiektu, wykorzystując techniki uzupełniania (ang. **inpaintingu**) oraz **sieci INR**.


### Czym jest inpainting?

**Inpainting** to technika stosowana w przetwarzaniu obrazów i filmów, która polega na **uzupełnianiu brakujących fragmentów** obrazu lub wideo w sposób jak najbardziej realistyczny. Nazwa pochodzi z języka angielskiego i oznacza dosłownie „domalowywanie”. Wykorzystuje się ją m.in. do usuwania obiektów ze zdjęć, rekonstrukcji zniszczonych fotografii czy naprawy uszkodzonych klatek filmowych.

Przykładowe użycie inpaintingu:


![](https://live.staticflickr.com/65535/54554097818_7395667a21.jpg)

W naszym zadaniu będziemy pracować nad bardziej zaawansowanym przypadkiem: **inpaintingiem dynamicznym**, czyli uzupełnianiem **brakujących fragmentów filmu w czasie**. Braki występują tylko w wybranym obszarze (np. środek klatki), który został wycięty we wszystkich klatkach filmu. Co ważne model nie musi generować całej klatki – jego zadaniem jest **tylko przewidzenie zawartości brakującego regionu**.

### Czym są sieci INR (Implicit Neural Representations)?

**INR (Implicit Neural Representations)** to nowoczesna technika reprezentowania danych przy użyciu sieci neuronowych. Tradycyjnie obrazy czy filmy przechowujemy jako macierze (siatki) wartości pikseli – każda komórka takiej macierzy to kolor piksela. Jednak INRy działają inaczej: zamiast zapisywać wartości pikseli bezpośrednio, sieć uczy się **funkcji**, która dla podanych współrzędnych przestrzennych (x, y) i – w naszym przypadku – także czasowej (t) zwraca przewidywane wartości.

Czyli zamiast przechowywać cały film, uczymy sieć, która działa jak „wirtualny projektor”:

$$
f(x, y, t) \rightarrow (R, G, B)
$$

Gdzie:

* $x, y$ – współrzędne piksela,
* $t$ – numer klatki filmu (czas),
* $R, G, B$ – kolor piksela w tej lokalizacji,

Taka reprezentacja ma wiele zalet:

* pozwala generować obraz w dowolnej rozdzielczości (nie ogranicza nas konkretna siatka pikseli),
* umożliwia interpolację pomiędzy klatkami (np. tworzenie płynnego ruchu),
* działa dobrze w zadaniach takich jak kompresja, super-rozdzielczość czy właśnie inpainting.

Dzieje się tak, ponieważ sieć nie ogranicza nas do wyboru wartości całkowitych, możemy wybierać dowolne wartości pośrednie np. (1.5, 44.5, 13.25).

Przykładowe wykorzystanie sieci INR pokazano na grafice:

![](https://live.staticflickr.com/65535/54555506837_48f1f8e589_b.jpg)

W naszym zadaniu model INR będzie reprezentował tylko **brakujący fragment filmu**, czyli obszar, którego nie znamy. Sieć musi nauczyć się nie tylko tego, **jak wygląda obraz w czasie**, ale także tego, **co się znajduje w niewidocznym obszarze**. To oznacza, że musi rozpoznać kontekst sceny – na podstawie otoczenia – i na tej podstawie realistycznie odtworzyć brakujące dane (zadanie *inpaintingu*).

### Czym jest maska segmentacji?

Druga część problemu dotyczy **segmentacji obrazu**. Segmentacja to proces rozpoznawania, które piksele należą do obiektu (np. postaci, samochodu, drzewa), a które do tła. Można to zapisać w postaci **maski** – obrazu (w naszym przypadku dla uproszczenia użyjemy maski binarnej), w którym każdy piksel ma wartość 0 (nie należy do obiektu) lub 1 (należy do obiektu). Dzięki temu można np. oddzielić postać od tła, co jest użyteczne w wielu zastosowaniach – od autonomicznych pojazdów po edycję wideo.

Na grafice przedstawiono przykładową maskę binarną:


![](https://live.staticflickr.com/65535/54554043929_118739686d_b.jpg)

Projektowany model będzie miał za zadanie przewidzieć **zarówno kolor piksela**, jak i jego **klasę (obiekt/tło)** – i to tylko dla brakującego obszaru. Dlatego też powyższą funkcję można rozszerzyć do postaci:

$$
f(x, y, t) \rightarrow (R, G, B, m)
$$

Gdzie:

* $x, y$ – współrzędne piksela,
* $t$ – numer klatki filmu (czas),
* $R, G, B$ – kolor piksela w tej lokalizacji,
* $m \in \{0,1\}$ – wartość maski segmentacyjnej.

## Zadanie

Twoim zadaniem jest zbudowanie sieci typu **INR** (Implicit Neural Representation), która przyjmuje jako wejście trzy liczby:

* **x** – współrzędna pozioma piksela,
* **y** – współrzędna pionowa piksela,
* **t** – numer klatki filmu.

> *Uwaga: Wartości wejściowe nie muszą być liczbami całkowitymi – sieć powinna działać również dla współrzędnych ciągłych, co umożliwia interpolację.*

Na wyjściu sieć powinna przewidywać:

* **wartości RGB** – czyli kolor piksela w danym momencie filmu,
* **wartość maski segmentacyjnej** – wartość 0 lub 1, która oznacza przynależność piksela do obiektu (0 – tło, 1 – obiekt).

### Dane

W zadaniu udostępniamy dwa zbiory danych:

* **Zbiór treningowy** – zawiera współrzędne punktów oraz odpowiadające im wartości RGB i maski, przeznaczony do nauki modelu,
* **Zbiór walidacyjny** – służy do oceny jakości predykcji modelu na niewidocznych wcześniej danych.

Dla wygody przygotowaliśmy gotowy **dataloader**, który dostarcza dane w postaci rekordów zawierających:

* współrzędne piksela: `(x, y, t)`,
* wartość RGB oryginalnego obrazu,
* wartość maski segmentacyjnej z zbioru `{0, 1}`.

Obrazy wejściowe mają rozdzielczość **256 × 256 pikseli**. Zbiory zawierają odpowiednio:

* **zbiór treningowy** – 59 obrazów,
* **zbiór walidacyjny** – 10 obrazów,
* **zbiór testowy** – 10 obrazów.

W zbiorze walidacyjnym udostępnione są wyłącznie współrzędne pikseli należących do **brakującego obszaru** – to właśnie ten region (o rozmiarze **64 × 64**) ma zostać *zrekonstruowany* przez model. Oznacza to, że model powinien nauczyć się uzupełniać brakujące fragmenty na podstawie wcześniej poznanych danych.

Twoje rozwiązanie zostanie ostatecznie przetestowane na Platformie Konkursowej na ukrytym zestawie danych testowych, który nie różni się znacząco od zbioru walidacyjnego pod względem rozkładu danych.

### Kryterium Oceny

Jak możesz się spodziewać, w ewaluacji będziemy oceniać dwa kluczowe aspekty Twojego rozwiązania:

- **Jakość rekonstrukcji obszaru brakującego obrazu** - jak dobrej jakości jest zwrócony obszar pikseli RGB, oraz ich spójność, ocenione zostanie to za pomocą metryki *PSNR*,
- **Dokładność predykcji binarnej maski** - jak dobrze udało się przewidzieć wartości maski w brakującym obszarze, ocenione zostanie to za pomocą dokładności klasyfikacji *acc*.

Przy definicji: **PSNR** (*peak signal-to-noise ratio*) - popluarna metryka jakości rekonstrukcji (np. obrazu).

Wartości *PSNR* i *acc* powstają z uśrednienia poszczególnych wartości na całym zbiorze testowym.

Finalny wynik definiujemy jako średnią ważoną tych dwóch aspektów:

$$ score = 0.7 \cdot P_{PSNR} \cdot 10 + 0.3 \cdot P_{acc} \cdot 10 $$

gdzie $P_{PSNR}$ to punkty przyznane za jakościowy wynik (*wartość metryki PSNR*) rozwiązania wynikający z progów:

$$ P_{PSNR} = \begin{cases}
0 & \text{jeżeli } PSNR < 14 \\
2 & \text{jeżeli } 14 <= PSNR < 17.1 \\
\frac{5}{4} \cdot PSNR - 19.375 & \text{jeżeli } 17.1 <= PSNR < 23.5 \\
10 & \text{jeżeli } 23.5 <= PSNR
\end{cases} $$

a $P_{acc}$ to punkty przyznane za dokładność maski (*wartość metryki acc*) wynikający z progów:


$$ P_{acc} = \begin{cases}
0 & \text{jeżeli } acc < 0.83 \\
\frac{20}{3} \cdot acc - \frac{83}{15} & \text{jeżeli } 0.83 <= acc < 0.98 \\
10 & \text{jeżeli } 0.98 <= acc
\end{cases} $$

Ta formuła wyraża, że aby uzyskać punkty, Twoje rozwiązanie musi osiągnąć minimalny wynik *PSNR* równy $14$ lub minimalny wynik *acc* równy $0.83$, a maksymalna liczba punktów ($100$) jest przyznawana za rozwiązania o wyniku wartości *PSNR* od $23.5$ (włącznie) i wartości *acc* od $0.98$ (włącznie).

> **Uwaga: Modele oparte na standardowych aktywacjach typu ReLU mogą nie wystarczyć.** \
> *Prosta sieć z aktywacjami ReLU osiąga jedynie około 5 punktów jakości rekonstrukcji (PSNR) na zbiorze walidacyjnym – aby uzyskać wyraźnie lepsze rezultaty, rozważ użycie aktywacji sinusoidalnych zgodnie z podejściem SIREN (Implicit Neural Representations with Periodic Activation Functions), https://arxiv.org/abs/2006.09661.*

Szczegóły implementacji tej formuły znajdziesz w funkcji `grade` w kodzie zadania.

## Ograniczenia

* Twoje rozwiazanie będzie testowane na Platformie Konkursowej.
* Model **nie** może korzystać z innych zbiorów danych oraz z pretrenowanych wag na innych zbiorach danych.
* Model może być uczony **maksymalnie** 6 minut z użyciem GPU.

## Pliki zgłoszeniowe

Ten notebook uzupełniony o Twoje rozwiązanie (patrz klasa `YourSolution`).

## Ewaluacja

Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`.

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów. Liczba punktów, którą zdobędziesz, będzie wyliczona na (tajnym) zbiorze testowym na Platformie Konkursowej na podstawie wyżej wspomnianego wzoru, zaokrąglona do liczby całkowitej. Jeśli Twoje rozwiązanie nie będzie spełniało powyższych kryteriów lub nie będzie wykonywać się prawidłowo, otrzymasz za zadanie 0 punktów.

Poniższa grafika ilustruje przykładowy proces inpaintingu z wykorzystaniem sieci INR:

![](https://live.staticflickr.com/65535/54554043974_246b208130_b.jpg)

## Kod Startowy

W tej sekcji inicjalizujemy środowisko poprzez zaimportowanie potrzebnych bibliotek i funkcji. Przygotowany kod ułatwi Tobie efektywne operowanie na danych i budowanie właściwego rozwiązania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# W czasie sprawdzania Twojego rozwiązania, wartość flagi FINAL_EVALUATION_MODE zostanie zmieniona na True
FINAL_EVALUATION_MODE = False

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

import json
import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from skimage.metrics import peak_signal_noise_ratio
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import tempfile
import tarfile

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# Ustawienie ziarna generatora liczb pseudolosowych w celu zapewnienia deterministycznych wyników.

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Ładowanie Danych

Za pomocą poniższego kodu wczytujemy dane.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

class ImageDataset(Dataset):
    """
    Klasa ImageDataset reprezentuje zbiór danych.
    Obsługuje obrazy i odpowiadające im maski, a także generuje współrzędne
    dla każdego piksela obrazu wraz z wartością czasową (t_value).
    """

    # Liczba klatek w filmie
    VIDEO_LENGTH = 79

    def __init__(
        self,
        img_dir: Path,
        mask_dir: Path,
        frame_names: list[str],
        mode: str,
        json_path: str | None = None,
    ):
        """
        Inicjalizuje obiekt klasy ImageDataset.

        Parametry:
        ----------
        img_dir : Path
            Ścieżka do katalogu z obrazami.
        mask_dir : str
            Ścieżka do katalogu z maskami.
        frame_names : list[str]
            Lista nazw plików obrazów (bez ścieżek).
        mode : str
            Tryb działania zbioru danych ("train" lub "val").
        json_path : str | None
            Ścieżka do pliku JSON z współrzędnymi prostokąta (domyślnie None).
        """
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.frame_names = frame_names
        self.mode = mode

        if mode not in ["train", "val"]:
            raise ValueError("Niepoprawny tryb. Użyj 'train' lub 'val'.")

        if mode == "val" and json_path is None:
            raise ValueError(
                "W trybie 'val' musisz podać ścieżkę do pliku JSON z współrzędnymi prostokąta."
            )

        if self.mode == "val":
            # Wczytanie współrzędnych brakującego prostokąta z pliku JSON
            with open(json_path, "r") as f:
                self.rect_coords = json.load(f)

    def __len__(self) -> int:
        """Zwraca liczbę ramek w zbiorze danych.

        Returns:
            Liczba ramek: int
        """
        return len(self.frame_names)

    def __getitem__(self, idx: int) -> tuple:
        """
        Zwraca dane dla podanego indeksu.

        Parameters
        ----------
        idx : int
            Indeks klatki.

        Returns
        -------
        Dict[str, Any]
            Słownik zawierający w zależności od trybu; (x, y, t),
            obrazy, maski i inne dane.
        """
        # Pobranie nazwy ramki
        frame_name = self.frame_names[idx]
        img_name = frame_name
        mask_name = frame_name.replace(".jpg", ".png")

        # Wczytanie obrazu i maski
        img = cv2.imread(os.path.join(self.img_dir, img_name))
        mask = cv2.imread(os.path.join(self.mask_dir, mask_name), cv2.IMREAD_GRAYSCALE)

        if self.mode == "train":
            # Wczytanie współrzędnych prostokąta i normalizacja
            h, w = img.shape[:2]
            img = img.astype("float32") / 255.0
            mask = mask.astype("float32") / 255.0

            # Generowanie siatki współrzędnych (x, y)
            x = torch.linspace(-1, 1, w)
            y = torch.linspace(-1, 1, h)
            grid_y, grid_x = torch.meshgrid(y, x, indexing="ij")
            coords = torch.stack([grid_x, grid_y], dim=-1).reshape(h * w, 2)

            # Obliczanie wartości czasowej (t_value)
            t_value = int(os.path.splitext(frame_name)[0])
            t_value = (t_value * 2) / (self.VIDEO_LENGTH - 1)
            t = torch.full((coords.shape[0], 1), t_value, dtype=torch.float32)
            coords = torch.cat([coords, t], dim=-1)

            # Spłaszczenie obrazu i maski do formatu per-pikselowego
            img = torch.tensor(img).view(-1, 3)
            mask = torch.tensor(mask).view(-1, 1)

            output = {
                "coordinates": coords,  # (x, y, t)
                "rgb": img,  # Wartości RGB prostokąta
                "mask": mask,  # Maska prostokąta
            }

        elif self.mode == "val":
            # Pobranie współrzędnych prostokąta
            x1, y1, x2, y2 = (
                self.rect_coords[frame_name]["x1"],
                self.rect_coords[frame_name]["y1"],
                self.rect_coords[frame_name]["x2"],
                self.rect_coords[frame_name]["y2"],
            )

            # Wymiary obrazu i maski
            h, w = x2 - x1, y2 - y1

            # Normalizacja obrazu i maski
            img = img.astype("float32") / 255.0
            mask = mask.astype("float32") / 255.0

            # Generowanie siatki współrzędnych (x, y) w obrębie prostokąta
            x = torch.linspace(x1 / img.shape[1] * 2 - 1, x2 / img.shape[1] * 2 - 1, w)
            y = torch.linspace(y1 / img.shape[0] * 2 - 1, y2 / img.shape[0] * 2 - 1, h)
            grid_y, grid_x = torch.meshgrid(y, x, indexing="ij")
            coords = torch.stack([grid_x, grid_y], dim=-1).reshape(h * w, 2)

            # Obliczenie wartości t_value
            t_value = int(os.path.splitext(frame_name)[0])
            t_value = (t_value * 2) / (self.VIDEO_LENGTH - 1)
            t = torch.full((coords.shape[0], 1), t_value, dtype=torch.float32)
            coords = torch.cat([coords, t], dim=-1)

            # Przekształcenie obrazu i maski do formatu tensorów
            img = torch.tensor(img).view(-1, 3)
            mask = torch.tensor(mask).view(-1, 1)

            output = {
                "coordinates": coords,  # (x, y, t)
                "original_image": img,  # Obraz oryginalny
                "original_mask": mask,  # Maska oryginalna
                "rectangle_coords": [x1, y1, x2, y2],  # Współrzędne prostokąta
            }

        return output

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################


def create_dataloaders(
    dir_name: Path, batch_size: int = 1, num_workers: int = 0
) -> tuple[DataLoader, DataLoader]:
    """
    Tworzy obiekty DataLoader dla zbiorów treningowego i walidacyjnego.

    Parameters
    ----------
    dir_name : Path
        Ścieżka do katalogu głównego zawierającego dane.
    batch_size : int, optional
        Rozmiar batcha dla loadera treningowego, domyślnie 1.
    num_workers : int, optional
        Liczba wątków używanych do ładowania danych, domyślnie 0.

    Returns
    -------
    tuple[DataLoader, DataLoader]
        Zwraca tuple zawierającą DataLoader dla zbioru treningowego, walidacyjnego.
    """
    # Ścieżka do zbioru treningowego
    train_path = Path(dir_name / "train")
    frame_names = sorted(os.listdir(train_path / "images"))
    train_dataset = ImageDataset(
        Path(train_path / "images"),
        Path(train_path / "masks"),
        frame_names,
        mode="train",
    )

    # Ścieżka do zbioru walidacyjnego
    val_path = Path(dir_name / "val")
    frame_names = sorted(os.listdir(val_path / "images"))
    val_dataset = ImageDataset(
        Path(val_path / "images"),
        Path(val_path / "masks"),
        frame_names,
        mode="val",
        json_path=val_path / "rectangles.json",
    )

    # Tworzenie DataLoaderów
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
    val_loader = DataLoader(
        val_dataset, batch_size=1, shuffle=False, num_workers=num_workers
    )

    return train_loader, val_loader

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
tempdir = tempfile.TemporaryDirectory()
TMP_DIR = tempdir.name
DATA_PATH = Path(TMP_DIR) / Path("data")

def unpack_tar_gz(filename: str, path: Path = DATA_PATH) -> None:
    """ Rozpakowuje archiwum tar.gz """
    with tarfile.open(filename, "r:gz") as tar:
        tar.extractall(path=path)
    
unpack_tar_gz("./train.tar.gz", DATA_PATH / Path("train"))
unpack_tar_gz("./val.tar.gz", DATA_PATH / Path("val"))
train_loader, val_loader = create_dataloaders(
    DATA_PATH, batch_size=2, num_workers=0
)

### Kod z Kryterium Oceniającym

Kod, zbliżony do poniższego, będzie używany do oceny rozwiązania na zbiorze testowym.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################


def grade(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    device: torch.device = "cuda",
) -> tuple[float, float, float]:
    """Ocena modelu na podstawie danych z DataLoadera.

    Funkcja oblicza średni PSNR (Peak Signal-to-Noise Ratio) oraz dokładność (accuracy)
    dla masek, a także zwraca końcową punktację.

    Parameters
    ----------
    model : torch.nn.Module
        Model do oceny.
    data_loader : torch.utils.data.DataLoader
        DataLoader zawierający dane do oceny.
    device : torch.device
        Urządzenie, na którym wykonywane są obliczenia (np. 'cuda' lub 'cpu').

    Returns
    -------
    tuple[float, float, float]
        Zwraca tuple zawierającą:
        - Punkty końcowe (float)
        - Średni PSNR (float)
        - Średnią dokładność (float)
    """
    model.eval()  # Ustawienie modelu w tryb ewaluacji
    psnr_list, acc_list = [], []  # Listy do przechowywania wyników PSNR i dokładności

    with torch.no_grad():  # Wyłączenie gradientów dla ewaluacji
        for batch_idx, batch in enumerate(data_loader):
            # Pobranie danych z batcha
            coordinates = batch["coordinates"].to(device)
            original_image = (
                batch["original_image"][0].cpu().numpy().reshape(256, 256, 3)
            )
            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
            original_mask = batch["original_mask"][0].cpu().numpy().reshape(256, 256)
            rect_coords = batch["rectangle_coords"]  # (x1, y1, x2, y2)

            # Przesłanie modelu na GPU
            model = model.cuda()
            rgb_pred, mask_pred = model(coordinates)
            rgb_pred = rgb_pred.squeeze(0).detach().cpu().numpy()
            rgb_pred = rgb_pred[:, [2, 1, 0]]
            mask_pred = mask_pred.squeeze(0).detach().cpu().numpy().squeeze()

            # Tworzenie obrazów zrekonstruowanych
            inpainted_img = original_image.copy()
            inpainted_mask = original_mask.copy()
            x1, y1, x2, y2 = map(int, rect_coords)
            idx = 0
            for y in range(y1, y2):
                for x in range(x1, x2):
                    inpainted_img[y, x, :] = rgb_pred[idx]
                    inpainted_mask[y, x] = mask_pred[idx]
                    idx += 1

            # Obliczanie PSNR dla regionu prostokąta
            gt_region = original_image[y1:y2, x1:x2, :]
            pred_region = inpainted_img[y1:y2, x1:x2, :]
            psnr = peak_signal_noise_ratio(gt_region, pred_region, data_range=1.0)
            psnr_list.append(psnr)

            # Obliczanie dokładności dla regionu maski
            gt_mask_region = original_mask[y1:y2, x1:x2]
            pred_mask_region = inpainted_mask[y1:y2, x1:x2]
            pred_mask_region = (pred_mask_region > 0.5).astype(np.uint8)
            acc = np.mean(pred_mask_region == gt_mask_region)
            acc_list.append(acc)

            # Wizualizacja wyników dla pierwszego batcha
            if batch_idx == 0:
                fig, axs = plt.subplots(1, 3, figsize=(15, 5))
                axs[0].imshow(original_image)
                axs[0].set_title("Original Image")
                axs[1].imshow(inpainted_img)
                axs[1].set_title("Inpainted Image")
                axs[2].imshow(inpainted_mask, cmap="gray")
                axs[2].set_title("Inpainted Mask")
                for ax in axs:
                    ax.axis("off")
                plt.tight_layout()
                plt.show()

    # Obliczanie średnich wyników PSNR i dokładności
    psnr_score = np.mean(psnr_list)
    acc_score = np.mean(acc_list)

    # Obliczanie punktacji na podstawie PSNR
    if psnr_score < 15.5:
        p_psnr = 0
    elif psnr_score < 23.5:
        p_psnr = (psnr_score - 15.5) * 5/4
    else:
        p_psnr = 10

    # Obliczanie punktacji na podstawie dokładności
    if acc_score < 0.83:
        p_acc = 0
    elif acc_score < 0.98:
        p_acc = (acc_score - 0.83) * 20/3
    else:
        p_acc = 1

    # Łączna punktacja
    points = 7 * p_psnr + 30 * p_acc

    return int(round(points, 0)), psnr_score, acc_score

### Przykładowe Rozwiązanie

Poniżej przedstawiamy uproszczone rozwiązanie, które służy jako przykład demonstrujący podstawową funkcjonalność notatnika.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

class DummyINR(nn.Module):
    """
    DummyINR to prosty model sieci neuronowej, który generuje różowe wartości RGB
    oraz maskę zerową na podstawie podanych współrzędnych wejściowych.
    """

    def __init__(self):
        """Inicjalizuje instancję klasy DummyINR."""
        super(DummyINR, self).__init__()

    def forward(self, coords):
        """
        Oblicza różowe wartości RGB oraz maskę zerową na podstawie współrzędnych wejściowych.

        Parametry:
        ----------
        coords : torch.Tensor
            Tensor wejściowy o kształcie (B, N, 3), gdzie B to rozmiar batcha,
            N to liczba punktów, a 3 to współrzędne (x, y, t).

        Zwraca:
        -------
        tuple:
            Dwie wartości zawierające:
                - rgb : torch.Tensor
                    Tensor różowych wartości RGB w zakresie [0, 1] o kształcie (B, N, 3).
                - mask : torch.Tensor
                    Tensor maski zerowej o kształcie (B, N, 1).
        """
        batch_size, num_points, _ = coords.shape

        pink_rgb = torch.tensor(
            [1, 0, 1], dtype=torch.float32, device=coords.device
        )
        rgb = pink_rgb.unsqueeze(0).unsqueeze(0).expand(batch_size, num_points, -1)  # Shape: (B, N, 3)

        # Mask values in the range [0, 1]
        mask = torch.zeros(
            (batch_size, num_points, 1), dtype=torch.float32, device=coords.device
        )  # Shape: (B, N, 1)

        return rgb, mask

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
    dummy_model = DummyINR()
    points, psnr, accuracy = grade(dummy_model, val_loader)
    print(f"Przykładowe rozwiązanie uzyskało: {points} pkt. na zbiorze walidacyjnym.")
    print(f"PSNR: {psnr:.2f}")
    print(f"Accuracy: {accuracy:.2f}")

## Twoje Rozwiązanie

W tej sekcji należy umieścić Twoje rozwiązanie. Wprowadzaj zmiany wyłącznie tutaj!

In [ ]:
class YourSolution(nn.Module):
    """Klasa YourSolution

    Klasa implementuje model sieci neuronowej INR, który przetwarza współrzędne
    wejściowe (x, y, t) i zwraca wartości RGB oraz maskę.

    Atrybuty:
    ----------
    Brak atrybutów do zainicjalizowania w konstruktorze.
    Model należy zdefiniować w metodzie `__init__`.
    """

    def __init__(self):
        """Inicjalizuje model YourSolution.

        Tutaj należy zdefiniować wszystkie warstwy i komponenty modelu.
        """
        super(YourSolution, self).__init__()
        # Inicjalizacja modelu
        pass

    def forward(self, coords):
        """Przetwarza współrzędne wejściowe i zwraca wyniki modelu.

        Parametry:
        ----------
        coords : torch.Tensor
            Tensor wejściowy o kształcie (B, N, 3), gdzie każda kolumna
            odpowiada współrzędnym (x, y, t).

        Zwraca:
        --------
        tuple
            Dwie wartości wyjściowe:
                - "rgb" : torch.Tensor
                    Tensor z wartościami RGB w zakresie [0, 1].
                - "mask" : torch.Tensor
                    Tensor z wartościami maski z zakresu [0, 1].
        """
        # Implementacja modelu
        pass



In [ ]:
def train(
    model,
    train_loader,
    epochs=1,
    lr=1000,
    device="cuda" if torch.cuda.is_available() else "cpu",
):
    """Funkcja trenująca model na podanym zbiorze danych.

    Parameters
    ----------
    model : torch.nn.Module
        Model, który ma zostać wytrenowany.
    train_loader : torch.utils.data.DataLoader
        DataLoader zawierający dane treningowe.
    epochs : int, optional
        Liczba epok treningowych (domyślnie 1).
    lr : float, optional
        Współczynnik uczenia (domyślnie 1000).
    device : str, optional
        Urządzenie, na którym ma być trenowany model (domyślnie "cuda", jeśli dostępne).

    Notes
    -----
    Funkcja wymaga dalszej implementacji pętli treningowej oraz logiki uczenia.
    """
    # Zbuduj pętlę uczącą
    model = model.to(device)
    optimizer = torch.optim.Adam(
        model.parameters(), lr=lr
    )  # Możesz zmienić optymalizator
    pass


In [ ]:
# Podaj liczbę epok, które chcesz przeprowadzić i wybrany współczynnik uczenia
EPOCHS = 40
LEARNING_RATE = 1e-4

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

model = YourSolution()
train(model, train_loader, epochs=EPOCHS, lr=LEARNING_RATE)

## Ewaluacja

Uruchomienie poniższej komórki pozwoli sprawdzić, ile punktów zdobyłoby Twoje rozwiązanie na danych walidacyjnych. Przed wysłaniem upewnij się, że cały notebook wykonuje się od początku do końca bez błędów w trybie *FINAL_EVALUATION_MODE = True* i bez konieczności ingerencji użytkownika po wybraniu opcji "Run All".

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
    points, psnr, accuracy = grade(model, val_loader)
    print(f"Twoje rozwiązanie uzyskało: {points} pkt. na zbiorze walidacyjnym.")
    print(f"PSNR: {psnr:.2f}")
    print(f"Dokładność klasyfikacji: {accuracy:.2f}%")

Podczas sprawdzania model zostanie zapisany jako `your_model.pkl` i oceniony na zbiorze testowym.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if FINAL_EVALUATION_MODE:
    import cloudpickle

    OUTPUT_PATH = "file_output"
    FUNCTION_FILENAME = "your_model.pkl"
    FUNCTION_OUTPUT_PATH = os.path.join(OUTPUT_PATH, FUNCTION_FILENAME)

    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    with open(FUNCTION_OUTPUT_PATH, "wb") as f:
        cloudpickle.dump(model, f)